In [1]:
cd ../../../

B:\Rico\Dev\GitHub\WebRico\src


In [2]:
from rico.backend import utils

In [237]:
class SuffixArray(list):
    
    def __init__(self, text, algo=u'radix sort'):
        
        # check if algorithm is valid
        
        algo = algo.lower()
        if algo not in SuffixArray.algorithms:
            raise ValueError(u"invalid argument: algo='%s' is not in %s" % (algo, str(SuffixArray.algorithms.keys())))
        
        self.text = text
        SuffixArray.algorithms[algo](self)
    
    def suffix(self, index):
        return u''.join(map(lambda num: '$' if num < 0 else chr(num), self.string[self[index]:]))
    
    def counting_sort(self):
        
        # transform text into list of integers
        string = transform(self.text)
        n = len(string)
        
        # create the initial list of indices
        list.__init__(self, range(n))
        position = string[:]
        value = [0] * n
        count = [0] * n
        
        # stable sort according to character index
        self.sort(cmp = lambda i, j: cmp(string[i], string[j]) if string[i] != string[j] else cmp(j, i))
        
        # commence counting sort
        gap = 1
        # resorting part
        while gap < (n << 1):
            value[self[0]] = 0
            for I in xrange(1, n):
                i, j = self[I - 1], self[I]
                value[j] = value[i] if position[i] == position[j] and i + gap < n and position[i + (gap >> 1)] == position[j + (gap >> 1)] else I
            for i in xrange(n):
                position[i] = value[i]
                value[i] = self[i]
                count[i] = i
            for i in xrange(n):
                index = value[i] - gap
                if index >= 0:
                    self[count[position[index]]] = index
                    count[position[index]] += 1
            gap <<= 1
        
        self.string = string
        self.position = position
        
        return self
    
    def radix_sort(self):
        
        # transform text into list of integers
        string = transform(self.text)
        n = len(string)
        
        # create the initial list of indices
        list.__init__(self, range(n))
        position = string[:]
        tmp = [0] * n
        gap = 1
        
        def compare(i, j):
            if position[i] != position[j]:
                return cmp(position[i], position[j])
            i += gap
            j += gap
            return cmp(position[i], position[j]) if i < n and j < n else cmp(j, i)
        
        while True:
            self.sort(cmp=compare)
            for i in xrange(1, n):
                tmp[i] = tmp[i - 1] + (1 if compare(self[i - 1], self[i]) < 0 else 0)
            for i in xrange(n):
                position[self[i]] = tmp[i]
            if tmp[n - 1] == n - 1:
                break
            gap <<= 1
        
        self.string = string
        self.position = position
        
        return self
    
    def dc3(self):
        
        # transform text into offsetted list of integers
        string = transform(self.text)
        n = len(string)
        origin = 256
        for letter in string:
            if letter >= 0 and letter < origin:
                origin = letter
        mn, mx = origin + min(string), max(string)
        
        list.__init__(self, [0] * n)
        
        # core dc3 algorithm
        sn = map(lambda x: x - mn + 1 if x >= 0 else origin + x - mn, string)
        sn.extend([0, 0, 0])
        DC3_algorithm(sn, self, n, mx - mn + 1)
        position = [0] * n
        
        for i in xrange(n):
            position[self[i]] = i
        
        self.string = string
        self.position = position
        
        pass
    
    algorithms = {
        u'counting sort': counting_sort,
        u'radix sort': radix_sort,
        u'dc3': dc3
    }


In [238]:
'''
Transforms a list or string into an array of integers separated by negative numbers.
Complexity: O(n)
'''
def transform(text):
    transformed_text = []
    
    if isinstance(text, list):
        for i, subtext in enumerate(text):
            if not utils.isstring(subtext):
                raise ValueError('text should be a string or a list of strings')
            transformed_text.extend(map(ord, subtext))
            transformed_text.append(~i)

    elif isinstance(text, unicode) or isinstance(text, str):
        transformed_text.extend(map(ord, text))
        transformed_text.append(-1)
    
    else:
        raise ValueError('text should be a string or a list of strings')

    return transformed_text

In [239]:
def DC3_radix_sort(a, b, r, R, n, K):
    c = [0] * (K + 1)
    for i in xrange(n):
        c[r[a[i] + R]] += 1
    sum = 0
    for i in xrange(K + 1):
        t = c[i]
        c[i] = sum
        sum += t
    for i in xrange(n):
        b[c[r[a[i] + R]]] = a[i]
        c[r[a[i] + R]] += 1

In [240]:
def DC3_algorithm(s, sa, n, K):
    # sizes
    nL, nM, nR = (n + 2) // 3, (n + 1) // 3, n // 3
    nLR = nL + nR
    
    # dummy subarrays
    sL = [0] * (nL + 10)
    saL = [0] * (nL + 10)
    sMR = [0] * (nLR + 13)
    saMR = [0] * (nLR + 13)
    
    # radix sort those not divisible by 3
    j = 0
    for i in xrange(n + nL - nM):
        if i % 3 != 0:
            sMR[j] = i
            j += 1
    
    # swap indices using radix sort
    DC3_radix_sort(sMR, saMR, s, 2, nLR, K)
    DC3_radix_sort(saMR, sMR, s, 1, nLR, K)
    DC3_radix_sort(sMR, saMR, s, 0, nLR, K)
    
    # assign bucket names
    name = 0
    c = [-1] * 3
    for i in xrange(nLR):
        equal = True
        for j in xrange(3):
            equal &= (c[j] == s[saMR[i] + j])
            c[j] = s[saMR[i] + j]
        if not equal:
            name += 1
        # key assignment
        if saMR[i] % 3 == 1:
            sMR[saMR[i] // 3] = name
        else:
            sMR[saMR[i] // 3 + nL] = name
    
    # ternary partition
    if name < nLR:
        DC3_algorithm(sMR, saMR, nLR, name)
        for i in xrange(nLR):
            sMR[saMR[i]] = i + 1
            i += 1
    else:
        for i in xrange(nLR):
            saMR[sMR[i] - 1] = i
    
    j = 0
    for i in xrange(nLR):
        if saMR[i] < nL:
            sL[j] = 3 * saMR[i]
            j += 1
    
    DC3_radix_sort(sL, saL, s, 0, nL, K)
    p, t = 0, nL - nM
    for k in xrange(n):
        i = (3 * saMR[t] + 1 if saMR[t] < nL else 3 * (saMR[t] - nL) + 2)
        j = saL[p]
        comp = (s[i], sMR[saMR[t] + nL]) <=(s[j], sMR[j // 3]) if saMR[t] < nL else (s[i], s[i + 1], sMR[saMR[t] - nL + 1]) <= (s[j], s[j + 1], sMR[j / 3 + nL])
        if comp:
            sa[k] = i
            t += 1
            if t == nLR:
                while p < nL:
                    k += 1
                    sa[k] = saL[p]
                    p += 1
        else:
            sa[k] = j
            p += 1
            if p == nL:
                while t < nLR:
                    k += 1
                    sa[k] = (3 * saMR[t] + 1 if saMR[t] < nL else 3 * (saMR[t] - nL) + 2)
                    t += 1

In [241]:
# Testing
trial_text = []

import cProfile, os, pstats
from random import randint

def statistics(algorithm):
    return os.path.join('rico', 'backend', 'notebook', 'suffix array ' + algorithm + ' stats')

def random_string(text_length):
    return ''.join([chr(randint(0, 25) + ord('a')) for i in xrange(text_length)])

def generate_text(trials=10, text_length=1000):
    print 'Trials:', trials
    print 'Text length:', text_length
    print ''
    global trial_text
    trial_text = [random_string(text_length) for i in xrange(trials)]

results = {}
def perform_algorithm(algorithm):
    global results
    results[algorithm] = []
    for text in trial_text:
        results[algorithm].append(SuffixArray(text, algo=algorithm))

def profiler(algorithm):
    cProfile.run('perform_algorithm("%s")' % algorithm, statistics(algorithm))
    print 'Algorithm:', algorithm
    pstats.Stats(statistics(algorithm)).strip_dirs().sort_stats(-1).print_stats()



In [242]:
generate_text(trials=10, text_length=100000)
profiler('dc3')
profiler('radix sort')

Trials: 10
Text length: 100000

Algorithm: dc3
Sun Nov 29 14:01:41 2015    rico\backend\notebook\suffix array dc3 stats

         1000273 function calls (1000263 primitive calls) in 6.820 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  1000010    0.096    0.000    0.096    0.000 <ipython-input-237-518b856d872b>:104(<lambda>)
       10    0.003    0.000    6.820    0.682 <ipython-input-237-518b856d872b>:3(__init__)
       10    0.397    0.040    6.817    0.682 <ipython-input-237-518b856d872b>:90(dc3)
       10    0.003    0.000    0.060    0.006 <ipython-input-238-7d5969a6e4e9>:5(transform)
       80    2.306    0.029    2.306    0.029 <ipython-input-239-2684ad35b5d6>:1(DC3_radix_sort)
    20/10    3.763    0.188    6.070    0.607 <ipython-input-240-bb226244ef67>:1(DC3_algorithm)
        1    0.000    0.000    6.820    6.820 <ipython-input-241-73047f640dac>:21(perform_algorithm)
        1    0.000    0.000    6.820    6.82

In [250]:
rs = results['radix sort'][0]
dc = results['dc3'][0]
for i in xrange(len(rs)):
    if rs[i] != dc[i]:
        print rs.suffix(i - 1)[:10]
        print rs.suffix(i)[:10]
        print rs.suffix(i + 1)[:10]
        print ''
        print dc.suffix(i - 1)[:10]
        print dc.suffix(i)[:10]
        print dc.suffix(i + 1)[:10]
        break

azzoqhvcpi
azzzbkvwln
baagtiufqa

azzoqhvcpi
aeczrlcjwa
azzzbkvwln
